# Time-Varying Effects Visualization

This notebook loads a model with time-varying effects and visualizes within-season form changes.

**Note**: Train models with time-varying effects using:
```bash
python train_model.py --model time-varying --data-dir ../Rugby-Data --save-as timevarying_model
```

In [ ]:
from rugby_ranking.notebook_utils import setup_notebook_environment, load_model_and_trace

# Setup: load data and configure plots
dataset, df, model_dir = setup_notebook_environment()

## 1. Load Time-Varying Model

In [2]:
# Load model checkpoint with time-varying effects
CHECKPOINT_NAME = "timevarying_model"  # Or "test_timevarying" from training tests

print(f"Loading checkpoint: {CHECKPOINT_NAME}")
model, trace = load_checkpoint(CHECKPOINT_NAME, verbose=True)

if not model.config.time_varying_effects:
    print("\n⚠️  Warning: This model was not trained with time-varying effects!")
    print("Train a new model with: python train_model.py --model time-varying")
else:
    print("\n✓ Model includes time-varying effects")
    print(f"  Player trend SD: {model.config.player_trend_sd}")
    print(f"  Team trend SD: {model.config.team_trend_sd}")

Loading checkpoint: timevarying_model
Loading checkpoint: timevarying_model
✗ Failed to load checkpoint: Checkpoint not found: /home/daniel/.cache/rugby_ranking/timevarying_model


ValueError: Checkpoint not found: /home/daniel/.cache/rugby_ranking/timevarying_model

## 2. Load Data for Context

In [ ]:
# Load data for player/team lookups
DATA_DIR = Path("../../Rugby-Data")

dataset = MatchDataset(DATA_DIR, fuzzy_match_names=False)
dataset.load_json_files()
df = dataset.to_dataframe(played_only=True)

print(f"Data loaded: {len(df):,} observations")
print(f"Seasons in model: {len(model._season_ids)}")

## 3. Visualize Time-Varying Effects

In [ ]:
# Extract time-varying parameters from trace
if "beta_player_try_trend_raw" in trace.posterior:
    print("✓ Found player trend parameters")
    beta_trend = trace.posterior["beta_player_try_trend_raw"].values
    print(f"  Shape: {beta_trend.shape}")
else:
    print("⚠️  No player trend parameters found")

if "gamma_team_trend_raw" in trace.posterior:
    print("✓ Found team trend parameters")
    gamma_trend = trace.posterior["gamma_team_trend_raw"].values
    print(f"  Shape: {gamma_trend.shape}")
else:
    print("⚠️  No team trend parameters found")

## 4. Team Trajectory Visualization

In [ ]:
# Fit using Variational Inference (fast)
inference_config = InferenceConfig(
    vi_n_iterations=50000,
    vi_method="advi",
)

# Try to load checkpoint
try:
    fitter = ModelFitter.load("time_varying_model_v1", model)
    trace = fitter.trace
    print("Loaded checkpoint from disk")
except ValueError:
    fitter = ModelFitter(model, inference_config)

    print("Fitting time-varying model with VI (this may take 5-10 minutes)...")
    trace = fitter.fit_vi(n_samples=2000)
    print("Done!")

    # Save checkpoint
    checkpoint_path = fitter.save("time_varying_model_v1")
    print(f"Saved to: {checkpoint_path}")

## 4. Team Strength Over Time

Let's visualize how team strengths vary within seasons:

In [ ]:
def plot_team_trajectories(teams, season, score_type='tries'):
    """
    Plot within-season strength trajectories for selected teams.
    
    Shows base + trend effect from season start (t=0) to end (t=1).
    """
    score_idx = config.score_types.index(score_type)
    
    # Get posterior samples
    gamma_base_raw = trace.posterior['gamma_team_base_raw'].values
    gamma_trend_raw = trace.posterior['gamma_team_trend_raw'].values
    sigma_team = trace.posterior['sigma_team_base'].values
    lambda_team = trace.posterior['lambda_team'].values[:, :, score_idx]
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Time points within season
    t = np.linspace(0, 1, 50)
    
    for team in teams:
        # Find team-season index
        team_season_id = model._team_season_ids.get((team, season))
        if team_season_id is None:
            print(f"Warning: {team} not found in {season}")
            continue
        
        # Compute base and trend effects
        base = sigma_team * lambda_team * gamma_base_raw[:, :, team_season_id]
        trend = sigma_team * lambda_team * gamma_trend_raw[:, :, team_season_id]
        
        # Effect at each time point
        effects = base[:, :, None] + trend[:, :, None] * t
        
        # Mean and CI
        mean_effect = effects.mean(axis=(0, 1))
        lower = np.percentile(effects, 2.5, axis=(0, 1))
        upper = np.percentile(effects, 97.5, axis=(0, 1))
        
        # Plot
        ax.plot(t, mean_effect, label=team, linewidth=2.5)
        ax.fill_between(t, lower, upper, alpha=0.2)
    
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5, label='League average')
    ax.set_xlabel('Season Progress (0 = start, 1 = end)', fontsize=12)
    ax.set_ylabel(f'{score_type.capitalize()} Effect (log-rate)', fontsize=12)
    ax.set_title(f'Team Form Trajectories: {season}\n{score_type.capitalize()}', fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Example: Plot top teams from a recent season
season = recent_seasons[-1]
top_teams = ['Leinster', 'Munster', 'Ulster', 'Connacht', 'Glasgow Warriors']

print(f"Plotting team trajectories for {season}...")
plot_team_trajectories(top_teams, season, score_type='tries')

### Interpret the Trajectories

**Upward slopes**: Team improving as season progresses (momentum, gel time)

**Downward slopes**: Team declining (fatigue, injuries accumulating)

**Flat lines**: Consistent performance throughout season

## 5. Compare All Teams in a Season

Show base strength vs trend slope:

In [ ]:
def plot_base_vs_trend(season, score_type='tries', top_n=15):
    """
    Scatter plot: base strength vs trend slope.
    
    Quadrants:
    - Top-right: Strong team getting stronger
    - Top-left: Strong team declining  
    - Bottom-right: Weak team improving
    - Bottom-left: Weak team declining further
    """
    score_idx = config.score_types.index(score_type)
    
    # Get posterior samples
    gamma_base_raw = trace.posterior['gamma_team_base_raw'].values
    gamma_trend_raw = trace.posterior['gamma_team_trend_raw'].values
    sigma_team = trace.posterior['sigma_team_base'].values
    lambda_team = trace.posterior['lambda_team'].values[:, :, score_idx]
    
    # Compute for all team-seasons
    results = []
    for (team, szn), ts_id in model._team_season_ids.items():
        if szn != season:
            continue
        
        base_samples = sigma_team * lambda_team * gamma_base_raw[:, :, ts_id]
        trend_samples = sigma_team * lambda_team * gamma_trend_raw[:, :, ts_id]
        
        results.append({
            'team': team,
            'base_mean': base_samples.mean(),
            'base_std': base_samples.std(),
            'trend_mean': trend_samples.mean(),
            'trend_std': trend_samples.std(),
        })
    
    results_df = pd.DataFrame(results).sort_values('base_mean', ascending=False).head(top_n)
    
    # Plot
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Color by quadrant
    colors = []
    for _, row in results_df.iterrows():
        if row['base_mean'] > 0 and row['trend_mean'] > 0:
            colors.append('darkgreen')  # Strong + improving
        elif row['base_mean'] > 0 and row['trend_mean'] < 0:
            colors.append('orange')  # Strong + declining
        elif row['base_mean'] < 0 and row['trend_mean'] > 0:
            colors.append('lightblue')  # Weak + improving
        else:
            colors.append('red')  # Weak + declining
    
    ax.scatter(results_df['trend_mean'], results_df['base_mean'], 
               s=150, c=colors, alpha=0.7, edgecolors='black', linewidth=1.5)
    
    # Error bars
    ax.errorbar(results_df['trend_mean'], results_df['base_mean'],
                xerr=results_df['trend_std'], yerr=results_df['base_std'],
                fmt='none', ecolor='gray', alpha=0.3)
    
    # Labels
    for _, row in results_df.iterrows():
        ax.annotate(row['team'], (row['trend_mean'], row['base_mean']),
                   fontsize=9, ha='left', va='bottom', 
                   xytext=(5, 5), textcoords='offset points')
    
    # Reference lines
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
    
    ax.set_xlabel('Trend Slope (improvement/decline rate)', fontsize=12)
    ax.set_ylabel('Base Strength (season average)', fontsize=12)
    ax.set_title(f'Team Base Strength vs Form Trend: {season}\n{score_type.capitalize()}',
                fontsize=14, fontweight='bold')
    
    # Add quadrant labels
    ax.text(0.02, 0.98, 'Strong + Declining', transform=ax.transAxes,
           fontsize=10, va='top', ha='left', color='orange', fontweight='bold')
    ax.text(0.98, 0.98, 'Strong + Improving', transform=ax.transAxes,
           fontsize=10, va='top', ha='right', color='darkgreen', fontweight='bold')
    ax.text(0.02, 0.02, 'Weak + Declining', transform=ax.transAxes,
           fontsize=10, va='bottom', ha='left', color='red', fontweight='bold')
    ax.text(0.98, 0.02, 'Weak + Improving', transform=ax.transAxes,
           fontsize=10, va='bottom', ha='right', color='lightblue', fontweight='bold')
    
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return results_df

# Plot for most recent season
season = recent_seasons[-1]
print(f"Analyzing {season}...\n")
team_summary = plot_base_vs_trend(season, score_type='tries', top_n=20)

## 6. Player Form Trends

Similar analysis for players:

In [ ]:
def analyze_player_trends(player_name, season, score_type='tries'):
    """
    Show a specific player's base + trend for a season.
    """
    if player_name not in model._player_ids:
        print(f"Player '{player_name}' not found")
        return
    
    player_idx = model._player_ids[player_name]
    season_idx = model._season_ids[season]
    score_idx = config.score_types.index(score_type)
    
    # Get appropriate effect based on score type
    if score_type == 'tries':
        base_raw = trace.posterior['beta_player_try_base_raw'].values[:, :, player_idx, season_idx]
        trend_raw = trace.posterior['beta_player_try_trend_raw'].values[:, :, player_idx, season_idx]
        sigma = trace.posterior['sigma_player_try_base'].values
        sigma_trend = trace.posterior['sigma_player_try_trend'].values
        lambda_p = trace.posterior['lambda_player_try'].values[:, :, score_idx]
    else:
        base_raw = trace.posterior['beta_player_kick_base_raw'].values[:, :, player_idx, season_idx]
        trend_raw = trace.posterior['beta_player_kick_trend_raw'].values[:, :, player_idx, season_idx]
        sigma = trace.posterior['sigma_player_kick_base'].values
        sigma_trend = trace.posterior['sigma_player_kick_trend'].values
        lambda_p = trace.posterior['lambda_player_kick'].values[:, :, score_idx]
    
    # Compute effects
    base = sigma * lambda_p * base_raw
    trend = sigma_trend * lambda_p * trend_raw
    
    print(f"{'='*60}")
    print(f"  {player_name} - {season}")
    print(f"  {score_type.capitalize()}")
    print(f"{'='*60}")
    print(f"Base effect:  {base.mean():+.3f} ± {base.std():.3f}")
    print(f"Trend slope:  {trend.mean():+.3f} ± {trend.std():.3f}")
    print()
    
    # Interpret
    if abs(trend.mean()) < 0.05:
        print("Interpretation: Consistent form throughout season")
    elif trend.mean() > 0:
        print("Interpretation: Improving form (hot streak, gaining confidence)")
    else:
        print("Interpretation: Declining form (fatigue, injury, loss of form)")
    
    # Plot trajectory
    fig, ax = plt.subplots(figsize=(10, 5))
    t = np.linspace(0, 1, 50)
    
    effects = base[:, :, None] + trend[:, :, None] * t
    mean_effect = effects.mean(axis=(0, 1))
    lower = np.percentile(effects, 2.5, axis=(0, 1))
    upper = np.percentile(effects, 97.5, axis=(0, 1))
    
    ax.plot(t, mean_effect, linewidth=3, color='darkblue', label='Mean effect')
    ax.fill_between(t, lower, upper, alpha=0.3, color='lightblue', label='95% CI')
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5, label='League average')
    
    ax.set_xlabel('Season Progress (0 = start, 1 = end)', fontsize=12)
    ax.set_ylabel(f'{score_type.capitalize()} Effect (log-rate)', fontsize=12)
    ax.set_title(f'{player_name} Form Trajectory: {season}', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Example: Analyze a top player
analyze_player_trends('Antoine Dupont', recent_seasons[-1], score_type='tries')

## 7. Find Players with Biggest Trends

Who improved/declined the most?

In [ ]:
def find_extreme_trends(season, score_type='tries', top_n=10, min_matches=5):
    """
    Find players with biggest positive/negative trends in a season.
    """
    season_idx = model._season_ids[season]
    score_idx = config.score_types.index(score_type)
    
    # Filter players who played enough matches
    season_df = df_recent[df_recent['season'] == season]
    player_matches = season_df.groupby('player_name')['match_id'].nunique()
    qualified_players = player_matches[player_matches >= min_matches].index
    
    # Get appropriate trend parameters
    if score_type == 'tries':
        trend_raw = trace.posterior['beta_player_try_trend_raw'].values
        sigma_trend = trace.posterior['sigma_player_try_trend'].values
        lambda_p = trace.posterior['lambda_player_try'].values[:, :, score_idx]
    else:
        trend_raw = trace.posterior['beta_player_kick_trend_raw'].values
        sigma_trend = trace.posterior['sigma_player_kick_trend'].values
        lambda_p = trace.posterior['lambda_player_kick'].values[:, :, score_idx]
    
    # Compute trends for all players
    results = []
    for player, pid in model._player_ids.items():
        if player not in qualified_players:
            continue
        
        trend = sigma_trend * lambda_p * trend_raw[:, :, pid, season_idx]
        
        results.append({
            'player': player,
            'trend_mean': trend.mean(),
            'trend_std': trend.std(),
            'matches': player_matches[player],
        })
    
    results_df = pd.DataFrame(results)
    
    print(f"\n{score_type.upper()} - {season}")
    print(f"(Minimum {min_matches} matches)\n")
    
    print("TOP IMPROVERS (biggest positive trends):")
    print("-" * 60)
    top_improvers = results_df.nlargest(top_n, 'trend_mean')
    for i, row in enumerate(top_improvers.itertuples(), 1):
        print(f"{i:2d}. {row.player:30s} {row.trend_mean:+.3f} ± {row.trend_std:.3f} ({row.matches} matches)")
    
    print("\nBIGGEST DECLINERS (biggest negative trends):")
    print("-" * 60)
    top_decliners = results_df.nsmallest(top_n, 'trend_mean')
    for i, row in enumerate(top_decliners.itertuples(), 1):
        print(f"{i:2d}. {row.player:30s} {row.trend_mean:+.3f} ± {row.trend_std:.3f} ({row.matches} matches)")
    
    return results_df

# Find players with extreme trends
trend_analysis = find_extreme_trends(recent_seasons[-1], score_type='tries', top_n=10, min_matches=8)

## 8. Summary

The time-varying effects model reveals:

1. **Team dynamics**: Which teams build momentum vs fade
2. **Player form**: Who's hitting peak form vs struggling
3. **Seasonal patterns**: Early vs late season performance
4. **Prediction improvements**: Recent form weighted more heavily

### Next Steps

- Compare predictions: static vs time-varying model
- Validate on held-out matches
- Investigate specific cases (injuries, coaching changes)
- Extend to career-long trajectories (random walk across seasons)